In [259]:
import pandas as pd
import numpy as np
from scipy import stats
#testserdar

In [135]:
wsp_contract = pd.read_excel("../../Datasets/study-1-data.xlsx")



In [136]:
wsp_contract['Sales'] = wsp_contract[['Demand', 'Stock']].min(axis=1)

In [137]:
wsp_contract['Leftovers'] = wsp_contract['Stock'] - wsp_contract['Sales']

In [138]:
wsp_contract['Unmet_Demand'] = wsp_contract['Demand'] - wsp_contract['Sales']

In [139]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Unnamed: 5,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,NaN,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,NaN,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,NaN,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,NaN,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,NaN,Retailer 1,60,60,0,75


In [140]:
wsp_contract.drop(columns=["Unnamed: 5"], inplace=True)

In [141]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75


In [142]:
wsp_contract['Realized_Mfg_Profit'] = (wsp_contract['Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Stock']) 

In [143]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300


In [144]:
wsp_contract['Realized_Retailer_Profit'] = (wsp_contract['Sales'] * 12) - (wsp_contract['Stock'] * wsp_contract['Wholesale p.'])

In [145]:
#wsp_contract['Total_Revenue'] = wsp_contract['Stock'] * wsp_contract['Wholesale p.']

In [146]:
wsp_contract['Realized_Mfg_Profit_Share'] = wsp_contract['Realized_Mfg_Profit'] / (wsp_contract['Realized_Mfg_Profit'] + wsp_contract['Realized_Retailer_Profit']) #wsp_contract.apply(lambda row: row['Realized_Mfg_Profit'] / row['Total_Manufacturer_Revenue'] if row['Total_Manufacturer_Revenue'] != 0 else 0, axis=1)

In [147]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Realized_Retailer_Profit,Realized_Mfg_Profit_Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,240,0.555556
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,400,0.444444
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,124,0.720721
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,153,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,240,0.555556


In [148]:

wsp_contract['Optimal_Stock'] = 100*((12-wsp_contract['Wholesale p.'])/12) + 50
wsp_contract['Optimal_Stock'] = wsp_contract['Optimal_Stock'].astype(int)

In [149]:
wsp_contract['Under/Overstock_wrt_Optimal'] = wsp_contract['Stock'] - wsp_contract['Optimal_Stock']

In [150]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Realized_Retailer_Profit,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,240,0.555556,83,-23
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,400,0.444444,91,-11
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,124,0.720721,91,-11
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,153,0.666667,75,-24
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,240,0.555556,83,-23


In [151]:
potential_demand = np.arange(50, 151)
expected_sales = []

for index, row in wsp_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

wsp_contract['Expected Sales'] = np.array(expected_sales)

In [152]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Realized_Retailer_Profit,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,240,0.555556,83,-23,59.455446
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,400,0.444444,91,-11,75.396040
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,124,0.720721,91,-11,75.396040
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,153,0.666667,75,-24,50.990099
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,240,0.555556,83,-23,59.455446
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,300,240,0.555556,83,-23,59.455446
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,0,0,NaN,58,-58,0.000000
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,306,153,0.666667,75,-24,50.990099
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,0,0,NaN,50,-50,0.000000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,300,240,0.555556,83,-23,59.455446


In [153]:
wsp_contract["Expected_Leftovers"] = wsp_contract["Optimal_Stock"] - wsp_contract["Expected Sales"]

In [154]:
wsp_contract['Expected Retailer Profit'] = (wsp_contract['Expected Sales'] * 12) - (wsp_contract['Wholesale p.'] * wsp_contract['Stock']) 

In [155]:
wsp_contract['Expected Mfg Profit'] = (wsp_contract['Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Stock'])

In [156]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Realized_Retailer_Profit,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,240,0.555556,83,-23,59.455446,23.544554,233.465347,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,400,0.444444,91,-11,75.396040,15.603960,344.752475,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,124,0.720721,91,-11,75.396040,15.603960,344.752475,320
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,153,0.666667,75,-24,50.990099,24.009901,152.881188,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,240,0.555556,83,-23,59.455446,23.544554,233.465347,300


In [157]:
wsp_contract['Expected Mfg. Profit Share'] = wsp_contract['Expected Mfg Profit'] / (wsp_contract['Expected Mfg Profit'] + wsp_contract['Expected Retailer Profit']) #wsp_contract.apply(lambda row: row['Expected Mfg Profit'] / row['Total Expected Revenue'] if row['Total Expected Revenue'] != 0 else 0, axis=1)

In [158]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Realized_Retailer_Profit,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,240,0.555556,83,-23,59.455446,23.544554,233.465347,300,0.562361
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,400,0.444444,91,-11,75.396040,15.603960,344.752475,320,0.481382
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,124,0.720721,91,-11,75.396040,15.603960,344.752475,320,0.481382
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,153,0.666667,75,-24,50.990099,24.009901,152.881188,306,0.666839
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,240,0.555556,83,-23,59.455446,23.544554,233.465347,300,0.562361
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,300,240,0.555556,83,-23,59.455446,23.544554,233.465347,300,0.562361
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,0,0,NaN,58,-58,0.000000,58.000000,0.000000,0,NaN
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,306,153,0.666667,75,-24,50.990099,24.009901,152.881188,306,0.666839
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,0,0,NaN,50,-50,0.000000,50.000000,0.000000,0,NaN
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,300,240,0.555556,83,-23,59.455446,23.544554,233.465347,300,0.562361


In [159]:
potential_demand = np.arange(51, 151)
expected_sales = []

for index, row in wsp_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Optimal_Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

wsp_contract['Predicted Sales'] = np.array(expected_sales)

In [160]:
#wsp_contract.to_csv("../../Datasets/study-1-data-wsp.csv", index=False)
#wsp_contract['Predicted Sales'] = wsp_contract[['Demand', 'Optimal_Stock']].min(axis=1).astype(int)

In [161]:
wsp_contract['Predicted Leftovers'] = wsp_contract['Optimal_Stock'] - wsp_contract['Predicted Sales']

In [162]:
wsp_contract['Predicted Retailer Profit'] = wsp_contract['Predicted Sales'] * 12 - (wsp_contract['Wholesale p.'] * wsp_contract['Optimal_Stock'])
wsp_contract['Predicted Mfg Profit'] = (wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Optimal_Stock'])

In [163]:
wsp_contract['Realized Retailer Profit'] = wsp_contract['Sales'] * 12 - (wsp_contract['Stock'] * wsp_contract['Wholesale p.'])

In [164]:
wsp_contract['Predicted Mfg. Profit Share'] = wsp_contract['Predicted Mfg Profit'] / (wsp_contract['Predicted Mfg Profit'] + wsp_contract['Predicted Retailer Profit'])

In [165]:
'''

wsp_contract['Predicted Mfg. Profit Share'] = wsp_contract.apply(
    lambda row: row['Predicted Mfg Profit'] / (row['Optimal_Stock'] * row['Wholesale p.']) if (row['Optimal_Stock'] * row['Wholesale p.']) != 0 else 0,
    axis=1
)
'''


"\n\nwsp_contract['Predicted Mfg. Profit Share'] = wsp_contract.apply(\n    lambda row: row['Predicted Mfg Profit'] / (row['Optimal_Stock'] * row['Wholesale p.']) if (row['Optimal_Stock'] * row['Wholesale p.']) != 0 else 0,\n    axis=1\n)\n"

In [166]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Realized Retailer Profit,Predicted Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,15.603960,344.752475,320,0.481382,82.80,8.20,356.60,364,400,0.505135
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,15.603960,344.752475,320,0.481382,82.80,8.20,356.60,364,124,0.505135
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,24.009901,152.881188,306,0.666839,72.00,3.00,189.00,450,153,0.704225
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,...,58.000000,0.000000,0,NaN,57.72,0.28,54.64,464,0,0.894648
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,...,24.009901,152.881188,306,0.666839,72.00,3.00,189.00,450,153,0.704225
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,...,50.000000,0.000000,0,NaN,50.00,0.00,0.00,450,0,1.000000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045


In [167]:
#wsp_contract.to_csv("../../Datasets/study-1-data-wsp.csv", index=False)

In [168]:
df1 = wsp_contract[0:40]

In [227]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Realized Retailer Profit,Predicted Mfg. Profit Share,null
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045,0
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,344.752475,320,0.481382,82.80,8.20,356.60,364,400,0.505135,0
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,344.752475,320,0.481382,82.80,8.20,356.60,364,124,0.505135,0
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,152.881188,306,0.666839,72.00,3.00,189.00,450,153,0.704225,0
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045,0


In [169]:
df1.tail()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Realized Retailer Profit,Predicted Mfg. Profit Share
35,1,36,Manufacturer 1,136,9,Retailer 1,0,0,0,136,...,75.000000,0.000000,0,NaN,72.00,3.00,189.00,450,0,0.704225
36,1,37,Manufacturer 1,116,8,Retailer 1,60,60,0,56,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
37,1,38,Manufacturer 1,105,8,Retailer 1,60,60,0,45,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
38,1,39,Manufacturer 1,80,8,Retailer 1,60,60,0,20,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
39,1,40,Manufacturer 1,75,8,Retailer 1,55,55,0,20,...,28.148515,218.217822,275,0.557563,77.72,5.28,268.64,415,220,0.607045


In [170]:
df2 = wsp_contract[wsp_contract['Player ID'] == 'Manufacturer 1']
df2.tail()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Realized Retailer Profit,Predicted Mfg. Profit Share
35,1,36,Manufacturer 1,136,9,Retailer 1,0,0,0,136,...,75.000000,0.000000,0,NaN,72.00,3.00,189.00,450,0,0.704225
36,1,37,Manufacturer 1,116,8,Retailer 1,60,60,0,56,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
37,1,38,Manufacturer 1,105,8,Retailer 1,60,60,0,45,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
38,1,39,Manufacturer 1,80,8,Retailer 1,60,60,0,20,...,23.544554,233.465347,300,0.562361,77.72,5.28,268.64,415,240,0.607045
39,1,40,Manufacturer 1,75,8,Retailer 1,55,55,0,20,...,28.148515,218.217822,275,0.557563,77.72,5.28,268.64,415,220,0.607045


In [171]:
wsp_contract = wsp_contract.fillna(0)
wsp_contract.to_csv('./curr_data.csv', index = False)

In [172]:
grouped_data_retailer = wsp_contract.groupby('Player ID')


In [176]:
wsp_contract = pd.read_csv("../../Notebooks/experiment/curr_data.csv")

In [272]:
#Null Hypothesis (H0): The mean order quantity made by retailers is not significantly different from the optimal stock level.

#Alternative Hypothesis (H1): The mean order quantity made by retailers is significantly different from the optimal stock level.

#Reasoning: Retailers are expected to be able to adapt correctly to the changing wholesale price as they are rational agents.

wsp_contract['null'] = 0
from scipy.stats import wilcoxon

print(stats.shapiro(wsp_contract["Stock"]))
print(stats.shapiro(wsp_contract["Optimal_Stock"]))
_, p_value = wilcoxon((wsp_contract["Stock"]), wsp_contract['Optimal_Stock'])
print("p value:",p_value)
if p_value < 0.05:
    print("Reject the null hypothesis: The median absolute difference is significantly different from zero.")
else:
    print("Fail to reject the null hypothesis: No significant difference from zero in the median absolute difference.")

ShapiroResult(statistic=0.9213677048683167, pvalue=5.289216595482103e-21)
ShapiroResult(statistic=0.9109364151954651, pvalue=2.791107770045616e-22)
p value: 2.7778300779822927e-06
Reject the null hypothesis: The median absolute difference is significantly different from zero.


In [264]:
my = []
i = 0
for group_key, group_data in grouped_data_retailer:
    a=group_data.iloc[0:5]['Expected Retailer Profit'].mean()
    b=group_data.iloc[20:35]['Expected Retailer Profit'].mean()
    my.append([a, b])
    i+=1

my = pd.DataFrame(my)


#hypo 2
#H0: The retailers' expected profits in the first 5 rounds are not significantly different compared to later stages of the game.
#H1: There is statistically significant difference between the first 5 round and the later stages of the game.
#Resoning: We expect retailers to learn how to efficiently play the game and increase their expected profits in the later stages of the game


print(stats.shapiro(my[0]))
print(stats.shapiro(my[1]))


t_stat, p_value = stats.ttest_rel(my[0], my[1])
print("P value is", p_value, "which is higher than 0.05.\nMeaning that the null hypothesis is accepted.")


ShapiroResult(statistic=0.9678138494491577, pvalue=0.6605005264282227)
ShapiroResult(statistic=0.9614971876144409, pvalue=0.5201285481452942)
P value is 0.43444818298325516 which is higher than 0.05.
Meaning that the null hypothesis is accepted.


In [179]:
#hypo 3: 

#H0: There is no statistically significant difference in the under/over ordering values of the best expected retailer profit scorers and the worst ones.
#meaning that the null hypothesis dictates that the retailers with highest expected retailer profits and lowest expected retailer profits have similar under/over ordering values
#Reasoning: We expect that the retailers whose expected profits are the highest among all should have come colser to the demand realization compared to the retailers with worst expected profits

liste = []
i = 0
for group_key, group_data in grouped_data_retailer:
    a=group_data['Expected Retailer Profit'].mean()
    liste.append([a, i])
    i+=1

liste = sorted(liste, key=lambda x: x[0], reverse=True)

liste 

[[382.2311881188119, 19],
 [343.444801980198, 17],
 [323.1596534653465, 4],
 [309.5851485148515, 3],
 [299.97995049504954, 12],
 [297.84158415841586, 9],
 [285.89826732673265, 20],
 [262.57252475247526, 6],
 [251.62896039603962, 16],
 [248.1269801980198, 15],
 [245.61707920792074, 13],
 [244.7628712871287, 10],
 [242.30915841584155, 11],
 [240.49529702970295, 7],
 [226.47920792079202, 0],
 [224.9918316831683, 5],
 [219.4126237623762, 18],
 [218.4272277227723, 8],
 [187.8868811881188, 1],
 [183.67376237623762, 14],
 [181.1641089108911, 2],
 [166.41707920792075, 21]]

In [267]:
first_indexes = [21, 1, 16, 18, 2]
last_indexes = [17, 0, 9, 5, 14]
df_best = pd.DataFrame()
df_worst = pd.DataFrame()

i = 0
for group_key, group_data in grouped_data_retailer:
    if i in first_indexes:
        df_best = pd.concat([df_best, group_data])
    elif i in last_indexes:
        df_worst = pd.concat([df_worst, group_data])
    i+=1
from scipy.stats import wilcoxon
print(stats.shapiro(abs(df_best['Under/Overstock_wrt_Optimal'])))
print(stats.shapiro(abs(df_worst['Under/Overstock_wrt_Optimal'])))


res = wilcoxon(abs(df_best['Under/Overstock_wrt_Optimal']) ,abs(df_worst['Under/Overstock_wrt_Optimal'])) 

print(res)
print("Null hypothesis is rejected")

ShapiroResult(statistic=0.926517903804779, pvalue=1.800900228943192e-08)
ShapiroResult(statistic=0.8850858211517334, pvalue=3.114175931018259e-11)
WilcoxonResult(statistic=7102.0, pvalue=0.0018774414138789553)
Null hypothesis is rejected


In [266]:

#hypothesis 4
#Retailers could send a signal to manufacturers by either ordering 0 stock or 51 stock in which case the profits are 0 or the lowest possible value other than zero
#Retailers do this when they want to express that the manufacturers' wholesale price offers are not acceptable by the retailers. This can affect the next rounds
#H0: The wholesale price offers of manufacturers are not significantly affected by the signals a retailer sends them the in the previous round.



main_df = pd.DataFrame()
for group_key, group_data in grouped_data_retailer:

    df_a = group_data[((group_data['Stock'] == 0 ) | (group_data['Stock'] == 51 )) & (group_data.index != group_data.index[-1]) ]
    indices = df_a.index + 1
    df_b =wsp_contract.iloc[indices]
    df_a = df_a.reset_index(drop =True)
    df_b = df_b.reset_index(drop =True)
    df_c = pd.concat([df_a['Wholesale p.'], df_b['Wholesale p.']], axis = 1)
    main_df = pd.concat([main_df, df_c],)

main_df.columns = ['a', 'b']

print(stats.shapiro(main_df['a']))
print(stats.shapiro(main_df['b']))


_, p_value = wilcoxon(main_df['a'], main_df['b'])
print(p_value ,"reject h0: signals do not affect the next turn's wholesale price offer, hypo 4")
print("We see in the end that when there is a signal by the retailer, the next round's wholesale price offer is affected by this decision")

ShapiroResult(statistic=0.890314519405365, pvalue=2.6951266107744232e-08)
ShapiroResult(statistic=0.9140936136245728, pvalue=5.084256144982646e-07)
1.3175845443509359e-06 reject h0: signals do not affect the next turn's wholesale price offer, hypo 4
We see in the end that when there is a signal by the retailer, the next round's wholesale price offer is affected by this decision


In [275]:
#hypo 5
# H0: There is no difference in means between the predicted Mfg. Profit Share and Expected Mfg Profit.

#reasoning: Predicted manufacturer profit relies on the retailer making optimal decisions. If they are making optimal decisions, then the difference of means of these datasets whould be 0.

#Result: P value is less than 0.05, rejecting the null hypothesis. Retailers do not make optimal decisions.
from scipy import stats

#Since p values of the shapiro tests are less than 0.05, these sets reject normality, so wilcoxon needs to be used instead of something like t-test.
print(stats.shapiro(wsp_contract['Predicted Mfg Profit']))
print(stats.shapiro(wsp_contract['Expected Mfg Profit']))


_, p_val = stats.wilcoxon(wsp_contract['Predicted Mfg Profit'], wsp_contract['Expected Mfg Profit'])

print(p_val, "\n", "Reject H0")

ShapiroResult(statistic=0.8194721341133118, pvalue=2.708517095198587e-30)
ShapiroResult(statistic=0.9152687788009644, pvalue=9.163949496737127e-22)
0.0008860172293888648 
 Reject H0
